In [1]:
import json, glob

In [2]:
import xgboost, textblob, string

import pandas as pd
import numpy as np

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

C:\Users\cranb\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Load json files

In [3]:
json_list = []

In [10]:
read_files = glob.glob("./crewler/*.json")
for f in read_files:
    with open(f, "rb") as infile:
        json_list.append(json.load(infile))

In [11]:
len(json_list)

282

In [12]:
json_list[1]

[{'description': 'ONE PIECE วันพีช เล่ม 88 ลูฟี่ปะทะกับคาตาคุริแห่งสามขุนพลอย่างดุเดือด ทว่าการโจมตีของเขากลับถูกอีกฝ่ายหลบเลี่ยงได้หมด จนต้องตกอยู่ในภาวะวิกฤติ!! แต่ถึงอย่างนั้น ลูฟี่ก็ไม่อาจปล่อยให้คาตาคุริไปหาพวกพ้องของตนได้ จึงตัดทางหนีทั้งหมดทิ้งแล้วเข้าต่อสู้อย่างสุดกำลัง แต่ทว่า...!? การ์ตูนผจญภัยกลางทะเลกว้างเพื่อตามหา One Piece!!',
  'link': 'https://www.naiin.com/product/detail/238651/',
  'title': 'ONE PIECE วันพีช เล่ม 88 ',
  'type': 'การ์ตูนความรู้'},
 {'description': 'ONE PIECE วันพีช เล่ม 87แผนการล้มเหลว!! แผนลอบสังหารของลูฟี่ เบจและซีซาร์ถูกขัดขวาง กองกำลังพันธมิตรจึงตกอยู่ในภาวะวิกฤติ!! พวกเขาไม่อาจต้านทานการโจมตีของบิ๊กมัมที่กำลังโกรธสุดขีดได้ จึงต้องรีบคิดหาทางหนีเป็นการด่วน แต่ทว่า...!? การ์ตูนผจญภัยกลางทะเลกว้างเพื่อตามหา One Piece!',
  'link': 'https://www.naiin.com/product/detail/238650/',
  'title': 'ONE PIECE วันพีช เล่ม 87',
  'type': 'การ์ตูนความรู้'},
 {'description': 'Ghost World คฤหาสน์ผีเฮี้ยน คำสัญญาแลกวิญญาณเรื่องราวของภูติผีปีศาจมีอยู่ทั่วทุกหนทุกแห่ง

# Convert to json object

In [13]:
json_data = []

In [14]:
for i in range(len(json_list)):
    for j in range(len(json_list[i])):
        json_data.append(json_list[i][j])

In [15]:
len(json_data)

22481

In [16]:
json_data[0]

{'description': 'ONE OF US IS LYING บอกแล้วไงว่าไม่ได้ฆ่าวัยรุ่นทุกคนต่างมีความลับเป็นของตัวเอง เรื่องที่อยากให้โลกลืมมันไปแล้วพวกเขาจะยอมแลกอะไรบ้าง เพื่อรักษาความลับนั้นไว้ด้วยชีวิตตนเอง หรือชีวิตของใครบางคนที่ล่วงรู้ เด็กนักเรียนมัธยมปลาย 5 คน ถูกกักบริเวณร่วมกันในห้องหนึ่ง แต่เมื่อสิ้นสุดเวลาทำโทษมีเพียง 4 คนที่รอดชีวิต.. บรอนวิน เด็กเนิร์ดอนาคตนักศึกษามหาวิทยาลัยผู้ไม่เคยแหกกฎ คูเปอร์ นักกีฬาเบสบอลดาวเด่นของโรงเรียนเนท แบดบอยเสเพล ชีวิตเคยพัวพันกับเรื่องค้ายาเสพติดแอ๊ดดี้ เจ้าหญิงงานพรอมผู้มีใบหน้าอันงดงามและไซมอน เจ้าของแอปฯ แฉเรื่องฉาวโฉ่ของเพื่อน ไซมอน เสียชีวิตก่อนจะได้โพสต์ความลับอันดำมืดของทุกคนลงทางออนไลน์ และตำรวจสรุปว่านี้ไม่ใช่อุบัติเหตุ นั่นทำให้เด็กทั้งสี่ที่เหลือต่างตกเป็นผู้ต้องสงสัยในคดีฆาตกรรมนี้ ว่าแต่ใครล่ะ คือผู้ลงมือฆ่า? แล้วใครจะเชื่อว่าทั้ง 4 คนนี้ไม่มีส่วนรู้เห็นด้วย? นี่คือนิยายที่จะพาคุณเข้าสู่โลกและความรู้สึกของวัยรุ่นยุคนี้ ในความสดใสมีชีวิตชีวาของพวกเขานั้นฉาบไปด้วยความมืดมนและการดิ้นรนที่จะเติบโตอย่างสมบูรณ์ หรืออย่างน้อยก็โตไปโดยที่ไม่ต้องฆ่าใครสักคน!

# Create a dataframe of category and description only

In [17]:
df = pd.DataFrame(json_data,columns=['type','description'])

# Delete the incomplete data

In [18]:
j = []
for i in range(len(df)):
    if df.iloc[i,1] == '':
        j.append(i)
df = df.drop(df.index[[j]])
# df_full_title = df_all_title.drop(df_all_title.index[[j]])

In [19]:
len(df)

14343

In [20]:
df

,type,description
0,นิยายแปล,ONE OF US IS LYING บอกแล้วไงว่าไม่ได้ฆ่าวัยรุ่...
1,การ์ตูนความรู้,ONE PIECE วันพีช เล่ม 88 ลูฟี่ปะทะกับคาตาคุริแ...
2,การ์ตูนความรู้,ONE PIECE วันพีช เล่ม 87แผนการล้มเหลว!! แผนลอบ...
3,การ์ตูนความรู้,Ghost World คฤหาสน์ผีเฮี้ยน คำสัญญาแลกวิญญาณเร...
4,การ์ตูนความรู้,GHOST WORLD คฤหาสน์ผีเฮี้ยน ปีศาจไร้หน้าเรื่อง...
5,การ์ตูนความรู้,โรงเรียนแมลงสุดเจ๋ง เล่ม 1 แมงมุมทารันทูล่ามาเ...
6,การ์ตูนความรู้,Kitty Candy Girls เล่ม 16 : โอ้โห! บริหารเวลาเ...
7,การ์ตูนความรู้,Kitty Candy Girls เล่ม 15 : โอ้โห! เป็นตัวของต...
8,การ์ตูนความรู้,แรบบิทไนท์ ทีมจันทราล่าสัตว์มหัศจรรย์ เล่มที่ ...
9,การ์ตูนความรู้,Little Princess ห้องเรียนฉบับคุณหนู : เปิดตำรา...


# Find the number of members for each category

In [21]:
grouped = df.groupby('type')
grouped.agg(np.size).sort_values(by='description', ascending=False)

,description
type,
หนังสือเด็กน้อย,965
จิตวิทยา,904
บริหาร,904
ธรรมะศาสนาและปรัชญา,894
ท่องเที่ยว,876
สุขภาพและความงาม,871
คอมพิวเตอร์/อินเตอร์เน็ต,855
นิยาย,852
วิชาการ/ความรู้,851


# Remove the categories that their members are less than 700

In [22]:
remove_list=['หนังสือต่างประเทศ']

In [23]:
for i in range(len(remove_list)):
    df = df[df.type!=remove_list[i]]

In [24]:
len(df)

13853

In [25]:
grouped = df.groupby('type')
grouped.agg(np.size).sort_values(by='description', ascending=False)

,description
type,
หนังสือเด็กน้อย,965
บริหาร,904
จิตวิทยา,904
ธรรมะศาสนาและปรัชญา,894
ท่องเที่ยว,876
สุขภาพและความงาม,871
คอมพิวเตอร์/อินเตอร์เน็ต,855
นิยาย,852
วิชาการ/ความรู้,851


# Train model
https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/

# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['description'], df['type'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(df['description'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(df['description'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x)
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x)

def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

# Naive Bayes on Character Level TF IDF Vectors
accuracy_nb = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("NB, CharLevel Vectors: {}".format(accuracy_nb))

# Linear Classifier on Character Level TF IDF Vectors
accuracy_lc = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("LR, CharLevel Vectors: {}".format(accuracy_lc))

# Extereme Gradient Boosting on Character Level TF IDF Vectors
accuracy_xg = train_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars.tocsc(), train_y, xvalid_tfidf_ngram_chars.tocsc())
print ("Xgb, CharLevel Vectors: {}".format(accuracy_nb))

def create_rnn():
    rnn = models.Sequential()
    rnn.add(layers.Embedding(5000,32,input_length=5000))
    rnn.add(layers.LSTM(32))
    rnn.add(layers.Dense(64,activation='elu'))
    rnn.add(layers.Dense(7,activation='softmax'))
    rnn.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy',metrics=['acc'])
    return rnn

classifier = create_rnn()
accuracy_rnn = train_model(classifier, xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, is_neural_net=True)
print("RNN, Charecter level: {}".format(accuracy_rnn))

def create_rnn_lstm():
    # Add an Input Layer
    input_layer = layers.Input((5000, ))

    # Add the word embedding Layer
    #embedding_layer = layers.Embedding(5000, 300)(input_layer)
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.LSTM(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rnn_lstm()
accuracy_rnn_lstm = train_model(classifier, xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, is_neural_net=True)
print("RNN-LSTM, Word Embedding: {}".format(accuracy_rnn_lstm))

def create_model_architecture(input_size):
    # create input layer 
    input_layer = layers.Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(100, activation="relu")(input_layer)
    
    # create output layer
    output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    return classifier 

classifier = create_model_architecture(xtrain_tfidf_ngram.shape[1])
accuracy = train_model(classifier, xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, is_neural_net=True)
print ("NN, Ngram Level TF IDF Vectors: {}".format(accuracy))

def create_cnn():
    # Add an Input Layer
    input_layer = layers.Input((5000, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(700, 300, trainable=True)(input_layer)
    #embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_cnn()
accuracy = train_model(classifier, xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, is_neural_net=True)
print ("CNN, Word Embeddings: {}".format(accuracy))

https://github.com/makcedward/nlp/blob/master/sample/nlp-character_embedding.ipynb

In [30]:
import re
import pandas as pd
import numpy as np
import sklearn
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Preprocessing
import nltk
from nltk.tokenize import sent_tokenize

# Modeling
import keras
from keras.models import Model, load_model
from keras.layers import Dense, Input, Dropout, MaxPooling1D, Conv1D, GlobalMaxPool1D, Bidirectional
from keras.layers import LSTM, Lambda, Bidirectional, concatenate, BatchNormalization, Embedding
from keras.layers import TimeDistributed
from keras.optimizers import Adam
import tensorflow as tf
import keras.backend as K

import IPython
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from sklearn.model_selection import train_test_split

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
train_df, test_df = train_test_split(df, test_size=0.1)
train_df, valid_df = train_test_split(train_df, test_size=0.1)

d={'type':['หนังสือพระราชนิพนธ์','นิยาย','คอมพิวเตอร์','ธรรมะศาสนาและปรัชญา','แม่และเด็ก','นิยาย','บ้านและสวน','โหราศาสตร์ดูดวงฮวงจุ้ย','บ้านและสวน','หนังสือท่องเที่ยว'],
   'description':['หนังสือประมวลพระบรมราโชวาท และพระราชดำรัส "คำพ่อสอน" เป็นหนังสือที่จัดพิมพ์ขึ้นเพื่อเผยแพร่แนวพระราชดำริของพระบาทสมเด็จพระปรมินทรมหาภูมิพลอดุยเดช รัชกาลที่ 9 เกี่ยวกับเศรษฐกิจพอเพียง ซึ่งประกอบด้วยมโนทัศน์ต่าง ๆ ไม่ว่าจะเป็นความพอเพียง ความพอประมาณ ความมีเหตุผล การมีภูมิคุ้มกัน รวมทั้งคุณธรรมและความรู้ ที่สำคัญคือความสามัคคี ซึ่งหากชาวไทยได้น้อมนำมาประพฤติปฏิบัติในการดำเนินชีวิตอย่างเข้มแข็งแล้ว จะทำให้เกิดความสุขร่วมกันอย่างยั่งยืนอีกด้วย',
                  'ต้าถังอันยิ่งใหญ่สถาปนาแผ่นดินขึ้นด้วยกำลังทหาร ให้ความสำคัญที่สุดคือวิชาบู๊ แม้นเจ้าเป็นเพียงไพร่ไร้เชื้อตระกูล หากพอมีพลังฝีมือ ไม่ขี้ขลาดหดหัว เจ้าก็นับว่ามีที่ให้ยืนในแผ่นดิน หนิงเชวีย เด็กกำพร้าเติบโตขึ้นในป่าเขา เมื่อเด็กชายเปลี่ยนกลายเป็นเด็กหนุ่ม มันก็เข้าร่วมเป็นพลทหารปลายแถวแห่งกองกำลังเล็กๆ สังกัดเมืองชายแดน ตัวมันที่ฝีมือไม่นับว่าเลวสร้างความดีความชอบมิใช่น้อย เป็นสุขอยู่ได้ไม่ต้องดิ้นรน ภายนอกดูไม่อินังขังขอบ ทว่าภายในใจกลับสุมซ่อนไว้ด้วยความลับ ความหลัง และความฝัน มันมีความลับ...ที่อาจต้องติดตัวมันไปจนตาย ด้วยมิอาจหาคนเข้าใจ มันมีความหลัง...ซึ่งเป็นแรงผลักดันให้มันมุ่งสู่เป้าหมาย มันมีความฝัน...ว่าสักวันจะได้กราบอาจารย์ ฝึกจิตบำเพ็ญฌาน และบัดนี้ หนทางสู่ความฝันนั้นได้เปิดออกเบื้องหน้ามันแล้ว!',
                 '"คู่มือช่างคอม 2019 ฉบับสมบูรณ์" เป็นหนังสือที่มีเนื้อหาข้อมูลอัดแน่นทั้งภาคทฤษฎีและปฏิบัติ สำหรับทั้งคอมพิวเตอร์พีซีและโน้ตบุ๊ค ในส่วนของทฤษฎี จะพูดถึงรายละเอียดและเทคโนโลยีของชิ้นส่วนต่างๆ ตลอดจนวิธีการจัดสเป็คและเลือกซื้อชิ้นส่วนต่างๆ เพื่อนำมาประกอบเป็นเครื่องพีซีด้วยตัวเอง โดยมีเนื้อหาละเอียด ครบถ้วน ทันสมัย และให้ข้อมูลที่ถูกต้องอัพเดตล่าสุด ในส่วนของปฏิบัติ จะพูดถึงการลงมือประกอบเครื่องคอมพิวเตอร์ด้วยตนเอง เริ่มตั้งแต่การจัดเตรียมอุปกรณ์ ลงมือประกอบจนเสร็จ ตั้งค่าไบออส จัดเตรียมฮาร์ดดิสก์ ติดตั้ง Windows 10 ติดตั้งไดรเวอร์และโปรแกรมใช้งานต่างๆ ที่จำเป็น ตลอดจนการตรวจสอบ ดูแล และแก้ไขปัญหาคอมพิวเตอร์ด้วยตนเอง โดยมีขั้นตอนอธิบายพร้อมภาพประกอบโดยละเอียด ชัดเจน และเข้าใจง่าย สามารถทำตามไปพร้อมกันได้ นอกจากนี้ยังได้เพิ่มเติมเนื้อหาที่เป็นประโยชน์อื่นๆ อาทิ การติดตั้ง Windows 7 และ 10 ไว้ในเครื่องเดียวกัน (UEFI + GPT) และการแก้ไขปัญหาการใช้งาน Windows 10 เป็นต้น เรียกได้ว่าเล่มเดียวจบครบถ้วนแน่นอน!',
                 ' หนังสือสวดมนต์แบบฮาวทูที่เน้นการสวดมนต์อย่างมีปัญญา เพื่อให้แต่ละวันของคุณสงบ สมดุล และราบรื่นมากขึ้น ทั้งยังช่วยแก้ปัญหาเฉพาะหน้าได้อย่างทันท่วงที ด้วยบทสวดมนต์สารพัดประโยชน์ที่สั้น กระชับ เหมาะสำหรับการนำไปใช้จริงในชีวิตประจำวัน โดยในเล่มประกอบด้วยบทสวดมนต์สั้น ๆ มากมายสำหรับทุกสถานการณ์ในชีวิต ไม่ว่าจะเป็น บทสวดมนต์พื้นฐานที่ชาวพุทธควรรู้ บทสวดมนต์สำหรับการเริ่มต้นวันใหม่ บทสวดมนต์สำหรับแก้ไขทุกปัญหาเฉพาะหน้าในชั่วโมงการทำงาน และอื่น ๆ อีกมากมายหวังว่าคุณจะพกหนังสือสวดมนต์เล่มนี้ไปทุกที่ และให้บทสวดต่าง ๆ ได้เป็นกัลยาณมิตรคอยช่วยเหลือคุณในทุก ๆ สถานการณ์ของชีวิต ไม่ว่าวันนั้นจะเป็นวันดีหรือวันร้าย ขอให้จำไว้ว่าคุณไม่ได้อยู่ตัวคนเดียว แต่ยังมี "สติ" และ "ปัญญา" เป็นเพื่อนคู่หูพร้อมจะช่วยเหลือคุณได้เสมอ เพียงแค่คุณเรียกมันออกมาด้วยเสียงสวดมนต์เท่านั้น',
                 'นำเสนอข้อมูลสำหรับการจัดการเรียนรู้เด็กปฐมวัยในรูปแบบโรงเรียนบ้าน โดยเน้นตามหลักสูตรการศึกษาปฐมวัย พุทธศักราช 2560 ของกระทรวงศึกษาธิการ ทั้งนี้เพื่อให้พ่อแม่ผู้ปกครองสามารถพัฒนาศักยภาพเด็กให้ถึงพร้อมตามวัย โดยจะให้ตัวอย่างการออกแบบจัดกิจกรรมและการเรียนรู้ที่พ่อแม่ผู้ปกครองต้องจัดทำ ร่วมกับการบูรณาการการเรียนรู้ในกิจกรรมประจำวัน จากตัวอย่างที่ให้นี้ ผู้ปกครองสามารถนำไปพัฒนาเป็นกิจกรรมการเรียนรู้เรื่องอื่น ๆ อย่างที่ต้องการได้ ที่สำคัญในการจัดกิจกรรมการเรียนรู้ที่บ้านนั้น พ่อแม่ผู้ปกครองต้องสร้างบรรยากาศการเรียนที่มีความรัก ความอบอุ่น และผ่อนคลายด้วย เพื่อให้ "โรงเรียนบ้าน" เป็นโรงเรียนที่งดงามน่าเรียนและเด็กสนุกที่จะเรียน การจัดการเรียนรู้ที่พ่อแม่จัดกระทำขึ้นก็จะประสบผลสำเร็จอย่างที่ต้องการอย่างแท้จริง คือ "เด็กเก่ง ดี และมีความสุขกับการเรียน" ',
                 ' "แฮร์รี่ พอตเตอร์" ไม่เคยได้ยินชื่อ "ฮอกวอตส์" มาก่อนด้วยซ้ำ ในวันที่จดหมายฉบับแรกเริ่มร่อนลงที่หน้าประตูบ้านเลขที่สี่ ซอยพรีเว็ต แฮร์รี่ได้เห็นเพียงกระดาษแข็งสีออกเหลือง หมึกสีเขียวมรกต พร้อมตราประทับสีม่วง ก่อนที่ลุงกับป้าจอมโหดจะยึดจดหมายไปอย่างรวดเร็ว แต่แล้วในวันเกิดปีที่สิบเอ็ดของแฮร์รี่ ชายร่างยักษ์ชื่อ "อรูเบอัส แฮกริด" ก็โผล่มาพร้อมข่าวน่าประหลาดใจ นั่นคือแฮร์รี่ พอตเตอร์เป็นพ่อมด และเขามีสิทธิ์เข้าเรียนที่โรงเรียน คาถาพ่อมดแม่และเวทมนตร์ศาสตร์ฮอกวอตส์! การผจญภัยแสนมหัศจรรย์กำลังจะเริ่มขึ้นแล้ว!',
                 'หนังสือเล่มนี้จะช่วยแนะนำยาสำหรับป้องกันและกำจัดโรคพืชและศัตรูของพืช เหมาะสำหรับผู้ปลูกต้นไม้ทุกกลุ่ม ตั้งแต่เกษตรกรไปจนถึงผู้รักต้นไม่ทั่วไป เพื่อให้ "รู้จักโรค รู้จักยา ขจัดการสูญเสียต้นไม้ที่มีค่าของเรา" โดยเนื้อหาจะกล่าวถึงสาเหตุและอาการโรคพืช เพื่อให้เกษตรกรและนักปลูกมือใหม่สามารถจับคู่สารเคมีป้องกันกำจัดศัตรูพืชได้อย่างถูกต้อง ซึ่งจะมีรายชื่อสารป้องกันกำจัดโรคพืชและศัตรูพืชที่ได้รับการขึ้นทะเบียนให้ในเล่ม พร้อมกับแนะนำการเลือกซื้อและใช้สารป้องกันกำจัดโรคพืช นอกจากนี้ ยังมีสูตรสารสกัดสมุนไพรที่มีฤทธิ์ป้องกันกำจัดโรคพืชและแมลงศัตรูพืชอีกด้วย',
                 '"คัมภีร์ปีกุน" เป็นหนังสือโหราศาสตร์ตามตำราจีน ว่าด้วยดวงชะตาปี 2562 ของคนในราศีต่าง ๆ ตั้งแต่เรื่องหน้าที่การงาน สุขภาพ วิธีผูกดวงชะตา สิ่งสิริมงคลต่าง ๆ ฯลฯ ตลอดจนถึงตำแหน่งทิศประจำปี การปรับดวง เสริมชะตา วิธีการไหว้บนบานศาลกล่าวของแต่ละราศี เพื่อความเป็นสิริมงคลของปีนักษัตรนั้น ๆ และในเล่มยังมียันฮู้ประกอบเพื่อไว้บูชาเพื่อความเป็นสิริมงคลอีกด้วย',
                 ' พบกับเรื่องน่ารู้และวิธีแก้ปัญหาต่าง ๆ เกี่ยวกับ "การปลูกเลี้ยงและดูแลต้นไม้" และ "งานโครงสร้างแบบเจาะลึก" โดยได้ผู้รู้ผู้เชี่ยวชาญอย่าง "คุณธราดล ทันด่วน" รุกขกรผู้มากประสบการณ์ เจ้าของฉายา "ครูต้อ หมอต้นไม้" , "คุณวรวิทย์ อังสุหัสต์" สถาปนิกที่หันมาเอาดีด้านจัดสวนมากว่า 30 ปี ให้คำแนะนำเรื่องงานโครงสร้าง "คุณวรวุฒิ แก้วสุก" นักจัดสวนที่ชำนาญการจัดสวนแนวทรอปิคัลและการทำน้ำตก "คุณศิริวิทย์ ริ้วบำรุง" และ "คุณจาตุรงค์ ขุนกอง" ผู้เชี่ยวชาญการจัดสวนอังกฤษ จาก Little Tree หัวข้อที่นำเสนอ อาทิ ปลูกไม้ใหญ่ใกล้บ้านไม่ต้องกังวลใจ ปลูกกุหลาบ รดน้ำอย่างถูกวิธีมีผลต่อรากพืช ต้นไม้ที่เป็นมิตรกับเพื่อนบ้าน การตัดแต่งต้นไม้ ประเภทต่าง ๆ ปัญหาบ่อร้าวบ่อรั่ว การวางหินให้ได้น้ำตกสวย เปลี่ยนปูนเป็นสวน การต่อเติมรั้ว และการต่อเติมห้องในสวน ฯลฯ เนื้อหาทั้ง 100 ข้อนี้ได้จากประสบการณ์ที่คลุกคลีอยู่กับสวนและต้นไม้ของแต่ละท่าน จึงเป็นคำตอบที่เชื่อถือได้และนำไปปฎิบัติได้จริง ยังมีเรื่องน่ารู้อีกมากมายที่คุณสามารถค้นหาคำตอบได้จาก หนังสือ "100 ความรู้คู่สวน" เล่มนี้ ซึ่งเปรียบเสมือนคู่มือที่ควรมีไว้คู่สวน เพื่อเตรียมรับมือกับปัญหาต่าง ๆ ที่อาจเกินขึ้นในสวนของคุณ',
                 'เที่ยวเต็มอิ่ม 5 วัน ในโซลครบทุกย่านดัง ก่อนเดินทางตะลุยรอบนอกอย่างเกาะนามิ หมู่บ้านฝรั่งเศส อินชอน ซูวอน และสนุกเอเวอร์แลนด์ แล้วค่อยลงใต้ไปปิดทริปที่ปูซาน 3 วันเต็ม กับทุกไฮไลต์ที่ไม่ควรพลาด เนื้อหาละเอียดอัดแน่น ทั้งข้อมูลการเดินทาง รายละเอียดสถานที่เที่ยว แผนที่ครบทุกย่าน ทุกเมือง ดูง่ายสบายตา ที่สำคัญคือใช้งานได้จริง!!']}
test_df=pd.DataFrame(data=d)

test_df

In [28]:
class CharCNN:
    
    CHAR_DICT = 'กขคงจฉชซฌญฎฏฐฑฒณดตถทธนบปผฝพฟภมยรลวศษสหฬอฮๅุูึฯี๊ัํะำไๆโเ้็่๋าใื์ิฺแ'
    
    def __init__(self, max_len_of_sentence, max_num_of_setnence, verbose=10):
        self.max_len_of_sentence = max_len_of_sentence
        self.max_num_of_setnence = max_num_of_setnence
        self.verbose = verbose
        
        self.num_of_char = 0
        self.num_of_label = 0
        self.unknown_label = ''
        
    def build_char_dictionary(self, char_dict=None, unknown_label='UNK'):
        """
            Define possbile char set. Using "UNK" if character does not exist in this set
        """ 
        
        if char_dict is None:
            char_dict = self.CHAR_DICT
            
        self.unknown_label = unknown_label

        chars = []

        for c in char_dict:
            chars.append(c)

        chars = list(set(chars))
        
        chars.insert(0, unknown_label) #inserts the element to the list.

        self.num_of_char = len(chars)
        self.char_indices = dict((c, i) for i, c in enumerate(chars))
        self.indices_char = dict((i, c) for i, c in enumerate(chars))
        
        if self.verbose > 5:
            print('Totoal number of chars:', self.num_of_char)

            print('First 3 char_indices sample:', {k: self.char_indices[k] for k in list(self.char_indices)[:3]})
            print('First 3 indices_char sample:', {k: self.indices_char[k] for k in list(self.indices_char)[:3]})
            

        return self.char_indices, self.indices_char, self.num_of_char
    
    def convert_labels(self, labels):
        """
            Convert label to numeric
        """
        self.label2indexes = dict((l, i) for i, l in enumerate(labels))
        self.index2labels = dict((i, l) for i, l in enumerate(labels))

        if self.verbose > 5:
            print('Label to Index: ', self.label2indexes)
            print('Index to Label: ', self.index2labels)
            
        self.num_of_label = len(self.label2indexes)

        return self.label2indexes, self.index2labels
    
    def _transform_raw_data(self, df, x_col, y_col, label2indexes=None, sample_size=None):
        """
            ##### Transform raw data to list
        """
        
        x = []
        y = []

        actual_max_sentence = 0
        
        if sample_size is None:
            sample_size = len(df)

        for i, row in df.head(sample_size).iterrows():
            x_data = row[x_col]
            y_data = row[y_col]

            sentences = sent_tokenize(x_data)
            x.append(sentences)

            if len(sentences) > actual_max_sentence:
                actual_max_sentence = len(sentences)

            y.append(label2indexes[y_data])

        if self.verbose > 5:
            print('Number of news: %d' % (len(x)))
            print('Actual max sentence: %d' % actual_max_sentence)

        return x, y
    
    def _transform_training_data(self, x_raw, y_raw, max_len_of_sentence=None, max_num_of_setnence=None):
        """
            ##### Transform preorcessed data to numpy
        """
        unknown_value = self.char_indices[self.unknown_label]
        
        x = np.ones((len(x_raw), max_num_of_setnence, max_len_of_sentence), dtype=np.int64) * unknown_value
        y = np.array(y_raw)
        
        if max_len_of_sentence is None:
            max_len_of_sentence = self.max_len_of_sentence
        if max_num_of_setnence is None:
            max_num_of_setnence = self.max_num_of_setnence

        for i, doc in enumerate(x_raw):
            for j, sentence in enumerate(doc):
                if j < max_num_of_setnence:
                    for t, char in enumerate(sentence[-max_len_of_sentence:]):
                        if char not in self.char_indices:
                            x[i, j, (max_len_of_sentence-1-t)] = self.char_indices['UNK']
                        else:
                            x[i, j, (max_len_of_sentence-1-t)] = self.char_indices[char]

        return x, y

    def _build_character_block(self, block, filters=[64, 100], kernel_size=[3, 3], 
                         pool_size=[2, 2], padding='valid', activation='relu', 
                         kernel_initializer='glorot_normal'):
        
        for i in range(len(filters)):
            block = Conv1D(
                filters=filters[i], kernel_size=kernel_size[i],
                padding=padding, activation=activation, kernel_initializer=kernel_initializer)(block)

        
        block = MaxPooling1D(pool_size=pool_size[i])(block)

        block = GlobalMaxPool1D()(block)
        block = Dense(128, activation='relu')(block)
        return block
    
    def _build_sentence_block(self, max_len_of_sentence, max_num_of_setnence, 
                              char_dimension=16,
                              filters=[[3, 5, 7], [200, 300, 300], [300, 400, 400]], 
#                               filters=[[100, 200, 200], [200, 300, 300], [300, 400, 400]], 
                              kernel_sizes=[[4, 3, 3], [5, 3, 3], [6, 3, 3]], 
                              pool_sizes=[[2, 2, 2], [2, 2, 2], [2, 2, 2]],
                             ):
        
        sent_input = Input(shape=(max_len_of_sentence, ), dtype='int64')
        embedded = Embedding(self.num_of_char, char_dimension, input_length=max_len_of_sentence)(sent_input)
        
        blocks = []
        for i, filter_layers in enumerate(filters):
            blocks.append(
                self._build_character_block(
                    block=embedded, filters=filters[i], kernel_size=kernel_sizes[i], pool_size=pool_sizes[i])
            )

        sent_output = concatenate(blocks, axis=-1)
        sent_encoder = Model(inputs=sent_input, outputs=sent_output)

        return sent_encoder
    
    def _build_document_block(self, sent_encoder, max_len_of_sentence, max_num_of_setnence, num_of_label,
                             loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy']):
        doc_input = Input(shape=(max_num_of_setnence, max_len_of_sentence), dtype='int64')
        doc_output = TimeDistributed(sent_encoder)(doc_input)

        doc_output = Bidirectional(LSTM(128, return_sequences=False))(doc_output)


        doc_output = Dense(128, activation='relu')(doc_output)
        doc_output = Dense(num_of_label, activation='sigmoid')(doc_output)

        doc_encoder = Model(inputs=doc_input, outputs=doc_output)
        doc_encoder.compile(loss=loss, optimizer=optimizer, metrics=metrics)
        return doc_encoder
    
    def preporcess(self, labels, char_dict=None, unknown_label='UNK'):
        if self.verbose > 3:
            print('-----> Stage: preprocess')
            
        self.build_char_dictionary(char_dict, unknown_label)
        self.convert_labels(labels)
    
    def process(self, df, x_col, y_col, max_len_of_sentence=None, max_num_of_setnence=None, label2indexes=None, sample_size=None):
        if self.verbose > 3:
            print('-----> Stage: process')
            
        if sample_size is None:
            sample_size = 1000
        if label2indexes is None:
            if self.label2indexes is None:
                raise Exception('Does not initalize label2indexes. Please invoke preprocess step first')
            label2indexes = self.label2indexes
        if max_len_of_sentence is None:
            max_len_of_sentence = self.max_len_of_sentence
        if max_num_of_setnence is None:
            max_num_of_setnence = self.max_num_of_setnence

        x_preprocess, y_preprocess = self._transform_raw_data(df=df, x_col=x_col, y_col=y_col, label2indexes=label2indexes)
        
        x_preprocess, y_preprocess = self._transform_training_data(x_raw=x_preprocess, y_raw=y_preprocess,
                                                                   max_len_of_sentence=max_len_of_sentence, 
                                                                   max_num_of_setnence=max_num_of_setnence)
        
        if self.verbose > 5:
            print('Shape: ', x_preprocess.shape, y_preprocess.shape)

        return x_preprocess, y_preprocess
    
    def build_model(self, char_dimension=16, display_summary=False, display_architecture=False, 
                    loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy']):
        if self.verbose > 3:
            print('-----> Stage: build model')
            
        sent_encoder = self._build_sentence_block(
            char_dimension=char_dimension,
            max_len_of_sentence=self.max_len_of_sentence, max_num_of_setnence=self.max_num_of_setnence)
                
        doc_encoder = self._build_document_block(
            sent_encoder=sent_encoder, num_of_label=self.num_of_label,
            max_len_of_sentence=self.max_len_of_sentence, max_num_of_setnence=self.max_num_of_setnence, 
            loss=loss, optimizer=optimizer, metrics=metrics)
        
        if display_architecture:
            print('Sentence Architecture')
            IPython.display.display(SVG(model_to_dot(sent_encoder).create(prog='dot', format='svg')))
            print()
            print('Document Architecture')
            IPython.display.display(SVG(model_to_dot(doc_encoder).create(prog='dot', format='svg')))
        
        if display_summary:
            print(doc_encoder.summary())
            
        
        self.model = {
            'sent_encoder': sent_encoder,
            'doc_encoder': doc_encoder
        }
        
        return doc_encoder
    
    def train(self, x_train, y_train, x_test, y_test, batch_size=128, epochs=1, shuffle=True):
        if self.verbose > 3:
            print('-----> Stage: train model')
            
        self.get_model().fit(
            x_train, y_train, validation_data=(x_test, y_test), 
            batch_size=batch_size, epochs=epochs, shuffle=shuffle)
        
#         return self.model['doc_encoder']

    def predict(self, x_test, return_prob=False):
        if self.verbose > 3:
            print('-----> Stage: predict')
            
        if return_prob:
            return self.get_model().predict(x_test)
        
        return self.get_model().predict(x_test).argmax(axis=-1)
    
    def get_model(self):
        return self.model['doc_encoder']

In [31]:
"""
    Maximum number of characters per sentence is 256.
    Maximum number of sentence is 5
"""

char_cnn = CharCNN(max_len_of_sentence=256, max_num_of_setnence=5)

"""
    First of all, we need to prepare meta information including character dictionary 
    and converting label from text to numeric (as keras support numeric input only).
"""
char_cnn.preporcess(labels=df['type'].unique())

"""
    We have to transform raw input training data and testing to numpy format for keras input
"""
x_train, y_train = char_cnn.process(df=train_df, x_col='description', y_col='type')
x_valid, y_valid = char_cnn.process(df=valid_df, x_col='description', y_col='type')
x_test, y_test = char_cnn.process(df=test_df, x_col='description', y_col='type')

#char_cnn.build_model()

-----> Stage: preprocess
Totoal number of chars: 68
First 3 char_indices sample: {'UNK': 0, 'ไ': 1, 'จ': 2}
First 3 indices_char sample: {0: 'UNK', 1: 'ไ', 2: 'จ'}
Label to Index:  {'นิยายแปล': 0, 'การ์ตูนความรู้': 1, 'คอมพิวเตอร์/อินเตอร์เน็ต': 2, 'งานอดิเรกและงานฝีมือ': 3, 'จิตวิทยา': 4, 'ดูดวง,โหราศาสตร์': 5, 'ท่องเที่ยว': 6, 'ธรรมะศาสนาและปรัชญา': 7, 'นิยาย': 8, 'บริหาร': 9, 'บ้านและสวน': 10, 'พระราชนิพนธ์': 11, 'วิชาการ/ความรู้': 12, 'สุขภาพและความงาม': 13, 'หนังสือเด็กน้อย': 14, 'อัตชีวประวัติ': 15, 'อาหาร': 16, 'เกษตรกรรม/สัตว์เลี้ยง': 17, 'แม่และเด็ก': 18}
Index to Label:  {0: 'นิยายแปล', 1: 'การ์ตูนความรู้', 2: 'คอมพิวเตอร์/อินเตอร์เน็ต', 3: 'งานอดิเรกและงานฝีมือ', 4: 'จิตวิทยา', 5: 'ดูดวง,โหราศาสตร์', 6: 'ท่องเที่ยว', 7: 'ธรรมะศาสนาและปรัชญา', 8: 'นิยาย', 9: 'บริหาร', 10: 'บ้านและสวน', 11: 'พระราชนิพนธ์', 12: 'วิชาการ/ความรู้', 13: 'สุขภาพและความงาม', 14: 'หนังสือเด็กน้อย', 15: 'อัตชีวประวัติ', 16: 'อาหาร', 17: 'เกษตรกรรม/สัตว์เลี้ยง', 18: 'แม่และเด็ก'}
-----> Stage: process


In [24]:
char_cnn.train(x_train, y_train, x_valid, y_valid, epochs=20)
char_cnn.get_model().save('./char_cnn_model.h5')

-----> Stage: train model
Train on 18486 samples, validate on 2054 samples
Epoch 1/20
18486/18486 [==============================] - 154s 8ms/step - loss: 2.4600 - acc: 0.1599 - val_loss: 2.3062 - val_acc: 0.2575
Epoch 2/20
18486/18486 [==============================] - 144s 8ms/step - loss: 2.0765 - acc: 0.3103 - val_loss: 1.8259 - val_acc: 0.3754
Epoch 3/20
18486/18486 [==============================] - 144s 8ms/step - loss: 1.6355 - acc: 0.4619 - val_loss: 2.0107 - val_acc: 0.4046
Epoch 4/20
18486/18486 [==============================] - 144s 8ms/step - loss: 1.2734 - acc: 0.5750 - val_loss: 1.0263 - val_acc: 0.6451
Epoch 5/20
18486/18486 [==============================] - 144s 8ms/step - loss: 0.9344 - acc: 0.6953 - val_loss: 0.6263 - val_acc: 0.7945
Epoch 6/20
18486/18486 [==============================] - 144s 8ms/step - loss: 0.6401 - acc: 0.7992 - val_loss: 0.5193 - val_acc: 0.8374
Epoch 7/20
18486/18486 [==============================] - 144s 8ms/step - loss: 0.3734 - acc: 0.8

x_test, y_test = char_cnn.process(df=test_df, x_col='description', y_col='type')

In [34]:
y_pre=char_cnn.predict(x_test)

-----> Stage: predict


In [35]:
y_pre

array([ 3,  6,  5, ..., 12,  1, 10], dtype=int64)

In [36]:
from sklearn.metrics import classification_report

In [37]:
print(classification_report(y_test,y_pre))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00       222
          1       0.98      1.00      0.99       187
          2       1.00      1.00      1.00        55
          3       0.99      0.98      0.98       172
          4       1.00      1.00      1.00       179
          5       1.00      1.00      1.00       206
          6       1.00      1.00      1.00       196
          7       1.00      1.00      1.00       145
          8       1.00      1.00      1.00       190
          9       1.00      1.00      1.00       202
         10       1.00      1.00      1.00       201
         11       1.00      1.00      1.00        64
         12       1.00      0.99      0.99       139
         13       1.00      1.00      1.00       125

avg / total       1.00      1.00      1.00      2283



In [ ]:
char_cnn_model_loaded.predict(x_test)